# パイプラインを構築する

これは、実験パイプライン構築を支援するためのノートブックです。  
パイプラインを構築することで、定められた順に実験を実行して実験を再現することが出来ます。

## ◆◆◆開発メモ◆◆◆
- 実験パイプラインの構築支援は、02_experimental_phaseとパッケージ内のコードipynbで行う
    - 02_experimental_phaseではパイプラインの構築(pipeline.yaml作成)の支援と、パッケージのコードに依存関係を記述する支援の2つのセクションを用意する。
    - また、パッケージ内のコードipynbでは、最上部に依存関係を記述するセクションを用意する。(Jupyterlab_templatesが使えれば、最上部にparametersタグをつきの依存関係記述用セルを設置したコード用jupyterlab_templatesを用意する)
    - pipeline.yamlの保存先をどこにするか また、pipeline.yamlは1つだと想定するのか

## 1. 前準備

パイプラインを構築するには、実行順を指定したい実験の全てのコードで依存関係を定義する必要があります。  
コードに依存関係を定義していない場合は、02_experimental_phaseの[依存関係を定義する](./define_experimental_dependencies.ipynb)セクションで依存関係を定義してください。

## 2. パイプライン設定ファイルを生成する

このステップでは、パイプライン全体の実行に必要な設定ファイルをシステムが作成します。以下のセルを実行してください。

In [ ]:
from ruamel.yaml import YAML
import os
import glob

files = glob.glob(os.getcwd()+'/*')
code_dict = {}

# YAMLのベースとなる辞書
yml = {'tasks': []}

for file in files:
    # ソースリストに載っている拡張子なら辞書作成する処理に修正したい
    if os.path.splitext(file)[1] == '.ipynb':
        code_dict['source']  = os.path.basename(file)
        code_dict['product'] = 'output/log_' + os.path.basename(file)
        yml['tasks'].append(code_dict)
        code_dict = {}

with open('pipeline.yaml', 'w') as file:
    yaml = YAML()
    yaml.default_flow_style = False
    yaml.dump(yml, file)

## 3. 変更内容を書き戻す

In [ ]:
import papermill as pm

pm.execute_notebook(
    '../util/datalad_save_push.ipynb',
    '-',
    parameters = dict(SAVE_MESSAGE = 'パイプライン構築', PATH = [未定])
)